# Using matrix interpolation to make smooth animations from an array of WCS images unevenly sampled in time

Arash Bahramian

## Framework
This is a very naive script, the steps are:
- Read in fits images
- Crop all of them centering on a specific sky location of interest and a spectific box size
- Extract the pixel-based 2-D array for each image and the timestamp associated with the image
- Interpolate between images using 1-D interpolation over time (1-D here means each pixel is interpolated in time between images independently of other pixels) and obtain a numerical function that can return a 2-D image at any time in the window of observations.
- Produce image frames for a linearly sampled time interval.
- Use Imagemagick to turn those images into a gif.


## Requirements

### In Python
Running in Python 3 using
- Numpy
- Scipy
- Matplotlib
- Astropy
- [CMasher](https://cmasher.readthedocs.io/) is also advised for perceptually-uniform colormaps, but not necessary.

### On your system
- [imagemagick](https://imagemagick.org/)
    
    You can install it in many linux systems with package managers, and I think with `brew` on Mac OS.
    
    Here, imagemagick is called via `os` operations from python.

In [1]:
# OS, operations
import os
import sys
import glob
import logging

# Scipy Ecosystem
import numpy as np
import scipy as sc
from scipy.interpolate import interp1d

# Astropy
import astropy
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata.utils import Cutout2D
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time

# Matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cmasher as cm
from matplotlib import rc

Loghandler = logging.getLogger(name='animator')

logging.basicConfig(level=logging.INFO)

Loghandler.info('Package versions')
Loghandler.info(f'\tPython:\t\t{sys.version}')
Loghandler.info(f'\tAstropy\t\t{astropy.__version__}')
Loghandler.info(f'\tMatplotlib\t{plt.matplotlib.__version__}')
Loghandler.info(f'\tNumpy\t\t{np.__version__}')
Loghandler.info(f'\tScipy\t\t{sc.__version__}')

INFO:animator:Package versions
INFO:animator:	Python:		3.10.5 | packaged by conda-forge | (main, Jun 14 2022, 07:06:46) [GCC 10.3.0]
INFO:animator:	Astropy		5.1
INFO:animator:	Matplotlib	3.5.2
INFO:animator:	Numpy		1.23.1
INFO:animator:	Scipy		1.8.1


## Main functions

In [2]:
def cut_and_interpolate(pathname_pattern, center_coords, box_size, interpolation_type='cubic'):
    """
    Function to read in fits images froma pathname pattern, crop them based on sky coord and interpolate in pixel coords
    Returns an array of time values (where actual data was captured) and a numerical function as a python object
    
    Assumes the input fits files have a common and sortable naming pattern. This is important for interpolation.
    
    The output function will be a function of time in units of days since first obs and returns an image for any t.
    """
    Loghandler.info(f'Finding input FITS files using pattern {pathname_pattern}')
    image_list = glob.glob(pathname_pattern)
    image_list.sort()
    
    center = center_coords
    box_reg = box_size
    
    obs_dates = []
    #obs_cut_hdu= []          # used for testing, but not really needed
    obs_cut_pix = []
    
    Loghandler.info('Starting the cropping loop')
    for img in image_list:
        # Capturing the timestamp info
        obs_dates.append(fits.open(img)[0].header['DATE-OBS'])
        # Cropping
        tmp_hdu =  fits.open(img)
        tmp_data = tmp_hdu[0].data[0][0]*1e3             # for ease of use going from Jy to mJy
        tmp_wcs = WCS(tmp_hdu[0].header, naxis=2)        # Depending on data format and obsevatory, you might need to tweak this line to extract appropriate axes
        tmp_cut = Cutout2D(tmp_data, center, box_reg, wcs=tmp_wcs)
        # Extract 2-D array image data for interpolation
        obs_cut_pix.append(tmp_cut.data)
    
    Loghandler.info('Interpolating')
    # For avoiding time madness, we just use delta-T from the first obs
    frame_time = (Time(obs_dates)-Time(obs_dates[0])).value
    interp_func = interp1d(frame_time,np.array(obs_cut_pix),axis=0,kind=interpolation_type)
    Loghandler.info('Cutting and interpolating done.')
    return frame_time, interp_func


def framemaker(frame_time, interp_func, box_size, physical_size, distance, nframes=50, dpi=150, save_dir='./output/', use_latex=True):
    """
    making jpeg frames and using imagemagick to turn those into a gif
    
    """
    # Matplotlib font and latex settings
    rc('text', usetex=use_latex)
    font = {'family' : 'serif',
            'weight' : 'bold',
            'size'   : '14'}
    rc('font', **font)
    
    Loghandler.info('Checking for images from a previous run in the output folder')
    if os.path.exists(OUTPUT_FOLDER+'frame001.jpg'):
        Loghandler.warning('Images files found.')
        Loghandler.warning('ImageMagick uses wildcards frame*jpg for finding images for animation.')
        Loghandler.warning('Images from the previous run that are not rewritten by the new run might be appended to the animation.')


    # calculating pixel scale and length scale
    Loghandler.warning('Length scale and pixel scale calculation are written haphazardly, make sure to sanity check versus DS9 or CARTA.')
    len_x, len_y = interp_func(0).shape
    pixel_scale = box_size[1].to(u.arcsec).value/len_x     # arcsec/pixel

    # The physical length scale transformed to fraction of the y-axis for ease of plotting
    ruler_fraction_len_y = (np.arctan((physical_size/distance).decompose()).to(u.arcsec)).value/(pixel_scale*len_y)
    
    Loghandler.info('Starting the interpolated imaging loop')
    # Making individual frame images for a linear sample of timesteps between the first and last obs
    for i,ti in enumerate(np.linspace(frame_time[0],frame_time[-1],nframes)):
        fig = plt.figure(figsize=(6,6),facecolor='k')
        ax = fig.add_subplot(1,1,1)
        
        # colormap settings, you might need to tweak things for your dataset
        # If you're not using cmasher, you can replace `cm.ember` with a matplotlib colormap, e.g., viridis
        cmap = cm.ember
        cmap.set_bad('black')
        cmap_norm = colors.PowerNorm(0.8, vmin=1.5, vmax=5.5)

        ax.imshow(interp_func(ti),
                  cmap = cmap,
                  norm = cmap_norm,
                  aspect = 'equal',
                  origin = 'lower',
                  interpolation = 'none')
        
        # adding the timestamp as the title
        # time is in days, for my case weeks was easier for demonstration
        ax.set_title(f'week {round(ti/7+1)}',color='w')
        
        # Adding rulers and scales
        # Given for interpolation we threw away the WCS info, it's just a rough approximation here
        # you need to make sure it is tailored to your case and data
        # I want show 2 ly on the image, in my images each pixel is roughly "pixel_scale" (defined above) arcsec across

        # Angular size ruler
        ax.hlines(1,5,5+(10/pixel_scale),color='w')
        ax.text(5+(5/pixel_scale),3,'10 arcsec',color='w',va='center',ha='center')
        
        # physical size ruler
        ax.vlines(len_x*0.95,len_y*(0.5-ruler_fraction_len_y/2),len_y*(0.5+ruler_fraction_len_y/2),color='w')
        ax.text(len_x*0.96,len_y*0.5,r'$\sim$2 light years at 3.4 kpc',color='w',rotation=270,va='center')
        
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.tick_params(axis='both', which='major', length=0)
    
        
        if i+1 < 10:
            fig.savefig(f'{save_dir}frame00{i+1}.jpg',dpi=dpi, bbox_inches='tight')
        elif i+1 < 100:
            fig.savefig(f'{save_dir}frame0{i+1}.jpg',dpi=dpi, bbox_inches='tight')
        else:
            fig.savefig(f'{save_dir}frame{i+1}.jpg',dpi=dpi, bbox_inches='tight')    
        plt.close(fig)

    # The imagemagick call
    # Note that the delay time is in 1/100th of a second
    # See details of options here: https://www.imagemagick.org/Usage/anim_basics/
    Loghandler.info('Calling Imagemagick')
    os.system(f'convert -delay 5 -loop 0 {save_dir}frame*jpg {save_dir}animation.gif')
    Loghandler.info('Frames and animation done.')
    return

## Execution

In [3]:
INPUT_PATTERN = 'input/*'
OUTPUT_FOLDER = 'output/'

# Center of the field fot image
CENTER = SkyCoord(282.2077172*u.degree, -1.4971637*u.degree, frame='icrs')

# Size of the box to crop
BOX = [1.2*u.arcmin, 1.2*u.arcmin]   # first element is height, second element is width, see documentation for Cutout2D

FRAME_TIME, INTERP_F = cut_and_interpolate(INPUT_PATTERN, CENTER, BOX)
framemaker(FRAME_TIME, INTERP_F, BOX, physical_size=2*u.lightyear, distance=3.4*u.kpc)

INFO:animator:Finding input FITS files using pattern input/*
INFO:animator:Starting the cropping loop
INFO:animator:Interpolating
INFO:animator:Cutting and interpolating done.
INFO:animator:Checking for images from a previous run in the output folder
INFO:animator:Starting the interpolated imaging loop
INFO:animator:Calling Imagemagick
INFO:animator:Frames and animation done.
